In [ ]:
!pip install openai

In [ ]:
import json
from tqdm import tqdm
import random
import pandas as pd
import numpy as np
import re

In [ ]:
from openai import OpenAI
client = OpenAI(api_key="provide_your_api")

In [ ]:
fs_prompt = """You are an expert who works with theorem provers. Given some context and a question, generate the predicates and the first-order logic formula for contexts and question. Here is an example.
------
Context:
Anne is quiet. Erin is furry. Erin is green. Fiona is furry. Fiona is quiet. Fiona is red. Fiona is rough. Fiona is white. Harry is furry. Harry is quiet. Harry is white. Young people are furry. If Anne is quiet then Anne is red. Young, green people are rough. If someone is green then they are white. If someone is furry and quiet then they are white. If someone is young and white then they are rough. All red people are young.
Question:
Anne is white.
###
Predicates:
Quiet(x) ::: x is quiet
Furry(x) ::: x is furry
Green(x) ::: x is green
Red(x) ::: x is red
Rough(x) ::: x is rough
White(x) ::: x is white
Young(x) ::: x is young
Premises:
Quite(Anne) ::: Anne is quiet.
Furry(Erin) ::: Erin is furry.
Green(Erin) ::: Erin is green.
Furry(Fiona) ::: Fiona is furry.
Quite(Fiona) ::: Fiona is quiet.
Red(Fiona) ::: Fiona is red.
Rough(Fiona) ::: Fiona is rough.
White(Fiona) ::: Fiona is white.
Furry(Harry) ::: Harry is furry.
Quite(Harry) ::: Harry is quiet.
White(Harry) ::: Harry is white.
∀x (Young(x) → Furry(x)) ::: Young people are furry.
Quite(Anne) → ¬Red(Anne) ::: If Anne is quiet then Anne is not red.
∀x (Young(x) → Rough(x)) ::: Young, green people are rough.
∀x (Green(x) → Rough(x)) ::: Young, green people are rough.
∀x (Green(x) → White(x)) ::: If someone is green then they are white.
∀x (Furry(x) ∧ Quite(x) → ¬White(x)) ::: If someone is furry and quiet then they are not white.
∀x (Young(x) ∧ White(x) → Rough(x)) ::: If someone is young and white then they are rough.
∀x (Red(x) → Young(x)) ::: All red people are young.
Conclusion:
White(Anne) ::: Anne is white.
------
Context:
[[PROBLEM]]
Question:
[[QUESTION]]
###
"""

In [ ]:
df_train = pd.read_json('proof_train_sampled_15000.json')

# Batch Generate

In [ ]:
# Function to create JSONL file for Chat Completions API
def create_jsonl_for_chat(df, output_file):
    with open(output_file, 'w') as file:
        for _, row in df.iterrows():
            context = row['Context'].strip()
            question = row['Question'].strip()
            full_prompt = fs_prompt.replace("[[PROBLEM]]", context).replace("[[QUESTION]]",question)
            json_line = {
                "custom_id": f"request-{row['id']}",
                "method": "POST",
                "url": "/v1/chat/completions",
                "body": {
                    "model": "gpt-4o",
                    "messages": [{"role": "user", "content": full_prompt}],
                    "max_tokens": 1000
                }
            }
            file.write(json.dumps(json_line) + '\n')

# Usage
create_jsonl_for_chat(df, 'proof_train_sampled_15000.json')


In [ ]:
batch_input_file = client.files.create(
  file=open("/content/folio_train_1001.jsonl", "rb"),
  purpose="batch"
)
batch_input_file_id = batch_input_file.id

In [ ]:
client.batches.create(
    input_file_id=batch_input_file_id,
    endpoint="/v1/chat/completions",
    completion_window="24h",
    metadata={
      "description": "folio fol generation 1001 records"
    }
)

In [ ]:
client.batches.retrieve("batch_id")

In [ ]:
content = client.files.content("output_file_id")

In [ ]:
content_bytes = content.read()
with open("save_file.jsonl", "wb") as file:
    file.write(content_bytes)

# Parse GPT output

In [ ]:
gpt_op = {}
# lines = content.strip().split('\n')
with open('batch_FBKUJMbX3iZcL4GS6kPVQnsg_output.jsonl', 'r') as file:
  # Parse each line as JSON
  for line in file:
      data = json.loads(line)
      # Now 'data' contains the parsed JSON object for each line
      gpt_op[data['custom_id']] = data['response']['body']['choices'][0]['message']['content']

In [ ]:
df_logic = pd.DataFrame(list(gpt_op.items()), columns=['id', 'logic_program'])
df_logic['id'] = df_logic['id'].apply(lambda x: x.split("request-")[-1].strip())

In [ ]:
df_final = df_train.merge(df_logic, on='id')

In [ ]:
df_final.to_json("proof_train_logic.json", orient="records", indent=4)

Pass this to the tool to extract valid results.